# Guided Capstone Step 4. Pre-Processing and Training Data Development

**The Data Science Method**  


1.   Problem Identification 


2.   Data Wrangling 
  
 
3.   Exploratory Data Analysis   

4.   **Pre-processing and Training Data Development**  
 * Create dummy or indicator features for categorical variables
  * Standardize the magnitude of numeric features
  * Split into testing and training datasets
  * Apply scaler to the testing set
5.   Modeling 
  * Fit Models with Training Data Set
  * Review Model Outcomes — Iterate over additional models as needed.
  * Identify the Final Model

6.   Documentation
  * Review the Results
  * Present and share your findings - storytelling
  * Finalize Code 
  * Finalize Documentation

**<font color='teal'> Start by loading the necessary packages as we did in step 3 and printing out our current working directory just to confirm we are in the correct project directory. </font>**

In [1]:
pwd

'/Users/kev_weis/Desktop/GuidedCapstone2'

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import os as os
import matplotlib.pyplot as plt

**<font color='teal'>  Load the csv file you created in step 3, remember it should be saved inside your data subfolder and print the first five rows.</font>**

In [3]:
cd /Users/kev_weis/Desktop/GuidedCapstone2/data/


/Users/kev_weis/Desktop/GuidedCapstone2/data


In [4]:
step3data = pd.read_csv('step3_outputv3.csv')

In [5]:
df = pd.DataFrame(step3data)
df.head()

Name               Region       state  \
0               Hilltop Ski Area               Alaska      Alaska   
1            Sunrise Park Resort              Arizona     Arizona   
2  Yosemite Ski & Snowboard Area  Northern California  California   
3               Donner Ski Ranch        Sierra Nevada  California   
4                  June Mountain        Sierra Nevada  California   

   summit_elev  vertical_drop  trams  fastEight  fastSixes  fastQuads  quad  \
0         2090            294      0        0.0          0          0     0   
1        11100           1800      0        0.0          0          1     2   
2         7800            600      0        0.0          0          0     0   
3         8012            750      0        0.0          0          0     0   
4        10090           2590      0        0.0          0          2     0   

   ...  SkiableTerrain_ac  Snow Making_ac  daysOpenLastYear  yearsOpen  \
0  ...               30.0       30.000000        150.000000       36.0   
1  ...              800.0       80.000000        115.000000       49.0   
2  ...               88.0      174.873239        110.000000       84.0   
3  ...              505.0       60.000000        163.000000       82.0   
4  ...             1500.0      174.873239        115.103943       58.0   

   averageSnowfall  AdultWeekday  AdultWeekend  projectedDaysOpen  \
0             69.0     30.000000      34.00000              152.0   
1            250.0     74.000000      78.00000              104.0   
2            300.0     47.000000      47.00000              107.0   
3            400.0     75.000000      75.00000              170.0   
4            250.0     57.916957      64.16681              128.0   

   NightSkiing_ac  clusters  
0            30.0         2  
1            80.0         1  
2             0.0         1  
3             0.0         1  
4             0.0         1  

[5 rows x 27 columns]

## Create dummy features for categorical variables

**<font color='teal'> Create dummy variables for `state`. Add the dummies back to the dataframe and remove the original column for `state`. </font>**

Hint: you can see an example of how to execute this in Aiden's article on preprocessing [here](https://medium.com/@aiden.dataminer/the-data-science-method-dsm-pre-processing-and-training-data-development-fd2d75182967). 

In [6]:
dummy = pd.get_dummies(df['state'])

In [7]:
df = pd.concat([df, dummy], axis=1)

In [8]:
df = df.drop(['state', 'Region'], axis=1)

In [9]:
df.columns
df.shape

(167, 60)

## Standardize the magnitude of numeric features

**<font color='teal'> Using sklearn preprocessing standardize the scale of the features of the dataframe except the name of the resort which we done't need in the dataframe for modeling, so it can be droppped here as well. Also, we want to hold out our response variable(s) so we can have their true values available for model performance review. Let's set `AdultWeekend` to the y variable as our response for scaling and modeling. Later we will go back and consider the `AdultWeekday`, `dayOpenLastYear`, and `projectedDaysOpen`. For now leave them in the development dataframe. </font>**

In [10]:
# first we import the preprocessing package from the sklearn library
from sklearn.preprocessing import StandardScaler

# Declare an explanatory variable, called X,and assign it the result of dropping 'Name' and 'AdultWeekend' from the df
x = df.drop(['Name','AdultWeekend'], axis=1)

# Declare a response variable, called y, and assign it the AdultWeekend column of the df 
y = df.AdultWeekend

# Here we use the StandardScaler() method of the preprocessing package, and then call the fit() method with parameter X 
scaler = StandardScaler().fit(x)

# Declare a variable called X_scaled, and assign it the result of calling the transform() method with parameter X 
X_scaled=scaler.transform(x) 

In [11]:
print(x.head())
print(X_scaled)

   summit_elev  vertical_drop  trams  fastEight  fastSixes  fastQuads  quad  \
0         2090            294      0        0.0          0          0     0   
1        11100           1800      0        0.0          0          1     2   
2         7800            600      0        0.0          0          0     0   
3         8012            750      0        0.0          0          0     0   
4        10090           2590      0        0.0          0          2     0   

   triple  double  surface  ...  Rhode Island  South Dakota  Tennessee  Utah  \
0       1       0        2  ...             0             0          0     0   
1       3       1        0  ...             0             0          0     0   
2       1       3        1  ...             0             0          0     0   
3       1       5        2  ...             0             0          0     0   
4       0       4        1  ...             0             0          0     0   

   Vermont  Virginia  Washington  West Virgi

## Split into training and testing datasets

**<font color='teal'> Using sklearn model selection import train_test_split, and create a 75/25 split with the y = `AdultWeekend`. We will start by using the adult weekend ticket price as our response variable for modeling.</font>**

In [12]:
# Import the train_test_split function from the sklearn.model_selection utility.  
from sklearn.model_selection import train_test_split

# Get the 1-dimensional flattened array of our response variable y by calling the ravel() function on y
y = y.ravel()

# Call the train_test_split() function with the first two parameters set to X_scaled and y 
# Declare four variables, X_train, X_test, y_train and y_test separated by commas 
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.25, random_state=1)

In [13]:
y_train.mean()

56.290999283154115

Here we start the actual modeling work. First let's fit a multiple linear regression model to predict the `AdultWeekend` price.

# Guided Capstone Step 5. Modeling


This is the fifth step in the Data Science Method. In the previous steps you cleaned and prepared the datasets. Now it's time to get into the most exciting part: modeling! In this exercise, you'll build three different models and compare each model's performance. In the end, you'll choose the best model for demonstrating insights to Big Mountain management.



### **The Data Science Method**  


1.   Problem Identification 

2.   Data Wrangling 
  
3.   Exploratory Data Analysis 
 
4.   Pre-processing and Training Data Development

5.   **Modeling**
  * Fit Models with Training Data Set
  * Review Model Outcomes — Iterate over additional models as needed.
  * Identify the Final Model

6.   Documentation
  * Review the Results
  * Present and share your findings - storytelling
  * Finalize Code 
  * Finalize Documentation

## Fit Models with a Training Dataset

**<font color='teal'> Using sklearn, fit the model on your training dataset.</font>**

#### Model 1

In [14]:
#all first model set
from sklearn import linear_model
from sklearn.metrics import explained_variance_score,mean_absolute_error
lm = linear_model.LinearRegression()
model = lm.fit(X_train,y_train)

**<font color='teal'> Predict on the testing dataset and score the model performance with the y_test set and the y-pred values. The explained variance is a measure of the variation explained by the model. This is also known as the R-squared value. </font>**

Hint: you will have to use the `predict()` method here as it's used in this [DSM article](https://medium.com/@aiden.dataminer/the-data-science-method-dsm-modeling-56b4233cad1b) about modeling. 

In [15]:
# Make a variable called y_pred and assign it the result 
#of calling predict()on our model variable with 
#parameter X_test

y_pred = lm.predict(X_test)


## Review Model Outcomes — Iterate over additional models as needed

In [16]:
y_pred

array([5.12179871e+01, 5.11496277e+01, 5.07145691e+01, 6.41906433e+01,
       5.70109558e+01, 6.79914246e+01, 5.51320496e+01, 4.16232605e+01,
       4.06979675e+01, 6.83664246e+01, 5.57531433e+01, 4.53449402e+01,
       3.51808167e+01, 5.14289246e+01, 4.65412292e+01, 6.01828308e+01,
       4.02882996e+01, 7.02570496e+01, 7.35008850e+01, 1.07142553e+13,
       5.72746277e+01, 4.41706238e+01, 1.07136218e+13, 1.07142540e+13,
       6.31476746e+01, 6.01652527e+01, 5.68742371e+01, 4.81867371e+01,
       6.81476746e+01, 6.80363464e+01, 3.19367371e+01, 5.73761902e+01,
       6.57481995e+01, 7.11300964e+01, 6.17199402e+01, 3.89621277e+01,
       7.28439636e+01, 3.29748230e+01, 5.40436096e+01, 8.05041199e+01,
       5.62804871e+01, 4.09425964e+01])

In [17]:
# You might want to use the explained_variance_score() and mean_absolute_error() metrics.
# To do so, you will need to import them from sklearn.metrics. 
# You can plug y_test and y_pred into the functions to evaluate the model

In [18]:
evs_y = explained_variance_score(y_test, y_pred)

In [19]:
print(evs_y)


-4.449053971545804e+22


In [20]:
MAE = mean_absolute_error(y_test, y_pred)

In [21]:
print(MAE)

765288836778.1781


**<font color='teal'> Print the intercept value from the linear model. </font>**

Hint: our linear regression model `lm` has an attribute `intercept_` for the intercept

In [22]:
print(lm.intercept_)

192467851220.74924


**<font color='teal'> The intercept is the mean `AdultWeekend` price for all the resorts given the other characteristics. The addition or subtraction of each of the coefficient values in the regression are numeric adjustments applied to the intercept to provide a particular observation's value for the resulting `AdultWeekend` value. Also, because we took the time to scale our x values in the training data, we can compare each of the coeeficients for the features to determine the feature importances. Print the coefficient values from the linear model and sort in descending order to identify the top ten most important features.</font>** 


Hint: make sure to review the absolute value of the coefficients, because the adjustment may be positive or negative, but what we are looking for is the magnitude of impact on our response variable.

In [23]:
#You might want to make a pandas DataFrame displaying the coefficients for each state like so: 
df_coef = pd.DataFrame(abs(lm.coef_), x.columns, columns=['Coefficient'])


In [24]:
df_coef.head(10)

Coefficient
summit_elev    2.119866e+00
vertical_drop  6.521563e-01
trams          5.824153e+13
fastEight      5.620354e+13
fastSixes      1.116884e+13
fastQuads      2.756676e+12
quad           3.466525e+12
triple         5.732818e+12
double         6.717516e+12
surface        6.941497e+12

**<font color='teal'>You should see that the top ten important features are different states. However, the state is not something the managers at the Big Mountain Resort can do anything about. Given that we care more about actionable traits associated with ticket pricing, rebuild the model without the state features and compare the results. </font>**

Hint: Try to construct another model using exactly the steps we followed above. 

#### Model 2

**<font color='teal'> When reviewing our new model coefficients, we see `summit_elev` is now in the number two spot. This is also difficult to change from a management prespective and highly correlated with `base_elev` and `vertical_drop`.  This time, rebuild the model without the state features and without the `summit_elev` and without `base_elev`and compare the results. </font>**

#### Model 3

In [25]:
x3 = df.drop(['summit_elev'], axis=1).select_dtypes(include=np.number)

y = df.AdultWeekend

# Here we use the StandardScaler() method of the preprocessing package, and then call the fit() method with parameter X 
scaler3 = StandardScaler().fit(x)

# Declare a variable called X_scaled, and assign it the result of calling the transform() method with parameter X 
X_scaled3=scaler.transform(x3) 


In [26]:
y3 = y.ravel()

# Call the train_test_split() function with the first two parameters set to X_scaled and y 
# Declare four variables, X_train, X_test, y_train and y_test separated by commas 
X_train3, X_test3, y_train3, y_test3 = train_test_split(X_scaled3, y, test_size=0.25, random_state=1)

In [27]:
model3 = lm.fit(X_train3,y_train3)

In [28]:
y_pred3 = lm.predict(X_test3)

In [29]:
df_coef3 = pd.DataFrame(abs(lm.coef_), x3.columns, columns=['Coefficient'])

In [30]:
df_coef.head(10)

Coefficient
summit_elev    2.119866e+00
vertical_drop  6.521563e-01
trams          5.824153e+13
fastEight      5.620354e+13
fastSixes      1.116884e+13
fastQuads      2.756676e+12
quad           3.466525e+12
triple         5.732818e+12
double         6.717516e+12
surface        6.941497e+12

## Identify the Final Model

**<font color='teal'> Review the model performances in the table below and choose the best model for proving insights to Big Mountain management about what features are driving ski resort lift ticket prices. Type your choice in the final markdown cell — you will discuss this selection more in the next step of the guided casptone. </font>**

| Model | Explained Variance| Mean Absolute Error|Features Dropped|
| --- | --- | --- | --- |
| Model 1. | 0. |  |-|
| Model 2. | 0.| |'state'|
| Model 3. | 0. | |'state','summit_elev','base_elev'|

Model Selection:

In [31]:
print(df['AdultWeekend'].mean())

56.40643916468138
